In [2]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from shapely.geometry import Polygon
import math
from Visual import MapViser
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
import numpy as np

In [3]:
def get_crf(wacc, years):
    crf = (wacc * math.pow(1+wacc, years)) / (math.pow(1+wacc, years)-1)

    return round(crf,6)

In [ ]:
valid_cell = gpd.read_file('./data/res/valid_cell.gpkg')

In [6]:
aidc_power = 30 #MW

capex = 823690000

fom = 0.0065 * capex

fom_desert = 1.5 * fom

ele_cost_base = {
    'China':0.72, #yuan/kWh
    'Africa':1.4,
    'North America':1.68,
    'South America':1.155,
    'Europe':1.4,
    'Oceania':1.2,
    'India':0.91,
    'Asia':1.0
}

pue_base = 1.2

wacc_base = 0.075

building_yr = 20

In [ ]:
valid_cell['ele_price_base'] = -1.0

for k in ele_cost_base:
    valid_cell.loc[valid_cell['capex_tag']==k,'ele_price_base'] = ele_cost_base[k]
    
valid_cell['ele_cost_base'] = (valid_cell['ele_price_base'] *
                               aidc_power * 
                               pue_base *
                               0.75 * 
                               8760 * 
                               1000)


In [8]:
profit_scens = {'lcoe_mid':{'CapEx Rate':[],
                'PUE Rate':[],
                'WACC Rate':[],
                'Ele Potential':[],
                'Cap Potential':[]},
                
                'lcoe_low':{'CapEx Rate':[],
                'PUE Rate':[],
                'WACC Rate':[],
                'Ele Potential':[],
                'Cap Potential':[]},
                
                'lcoe_high':{'CapEx Rate':[],
                'PUE Rate':[],
                'WACC Rate':[],
                'Ele Potential':[],
                'Cap Potential':[]},
                
                'lcoe_freetrade':{'CapEx Rate':[],
                'PUE Rate':[],
                'WACC Rate':[],
                'Ele Potential':[],
                'Cap Potential':[]},
                
                'lcoe_2024':{'CapEx Rate':[],
                'PUE Rate':[],
                'WACC Rate':[],
                'Ele Potential':[],
                'Cap Potential':[]},
                
                'lcoe_freetrade':{'CapEx Rate':[],
                'PUE Rate':[],
                'WACC Rate':[],
                'Ele Potential':[],
                'Cap Potential':[]}
                }

In [ ]:
valid_cell['elecost_savings'] = -1.0

In [11]:
cr_range = np.arange(1,4.01,0.01)
pr_range = np.arange(1,4.01,0.01)
wr_range = np.arange(1,4)

for ky in profit_scens:
    for cr in cr_range:
        for pr in pr_range:
            for wr in wr_range:
                elecons_sens = aidc_power * pue_base * pr * 0.75 * 8760 * 1000
                
                exp_base = capex * get_crf(wacc=wacc_base*wr,years=building_yr) + fom
                exp_gap = capex * cr * get_crf(wacc=wacc_base*wr,years=building_yr) + fom_desert - exp_base
                
                valid_cell['elecost_savings'] = valid_cell['ele_cost_base'] - elecons_sens * valid_cell[ky]
                
                cap_count = valid_cell.loc[valid_cell['elecost_savings']>exp_gap]['equal_cap'].sum()
                ele_count = cap_count  * 8760 * 1e-6
                
                profit_scens[ky]['CapEx Rate'].append(cr)
                profit_scens[ky]['PUE Rate'].append(pr)
                profit_scens[ky]['WACC Rate'].append(wr)
                profit_scens[ky]['Ele Potential'].append(ele_count)
                profit_scens[ky]['Cap Potential'].append(cap_count)

In [16]:
for ky in profit_scens:
    profit_scens[ky] = pd.DataFrame.from_dict(profit_scens[ky])

In [20]:
import numpy as np

plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = 30

cmap = 'RdPu'
#cmap = 'BuGn'

for ky in profit_scens:

#for ky in ['lcoe_freetrade']:
    fig,axes = plt.subplots(1,3, figsize=(40,10),dpi=400)

    for idx,wacc in enumerate(wr_range):
        ax = axes[idx]
        vis = profit_scens[ky].loc[profit_scens[ky]['WACC Rate']==wacc]
        
        xi = pr_range
        yi = cr_range

        X, Y = np.meshgrid(xi, yi)
        
        Z = np.array(vis['Ele Potential']).reshape(len(xi),len(yi))
        
        contour = ax.contourf(X, Y, Z, 25, cmap=cmap)
        
        ax.set_title('WACC: ' +str(round(wacc * wacc_base*100,2))+'%')

        # 同时，我们可以叠加绘制等高线
        CS = ax.contour(X, Y, Z, 25, colors='grey', linewidths=0.75)

        # 6. 添加文字标记 (clabel)
        label_levels = np.arange(10, 240, 40)
        
        ax.clabel(CS, 
                  levels=label_levels,
                  inline=True,       # 只有 True 才能让线断开，字嵌进去
                  fontsize=25,        # 字号要小
                  colors='k',        # 文字颜色
                  manual=False       # 如果自动排版很乱，改为 True 手动鼠标点击放置
                )
        
        # 4. 添加颜色条、标签和标题
        plt.colorbar(contour, label='Electricity potential (PWh/yr)')
        
        ax.set_xlabel('CapEx (Non-IT) ratio to the real case')
        
        if idx == 0:
            ax.set_ylabel('PUE ratio to the real case')
        
        ax.set_xticks(np.arange(1.0,4.01,0.5),
                      [str(x)+'X' for x in np.arange(1.0,4.01,0.5)])
        
        ax.set_yticks(np.arange(1.0,4.01,0.5),
                      [str(x)+'X' for x in np.arange(1.0,4.01,0.5)])
        
        ax.minorticks_on()
                       
        ax.tick_params(axis='y', 
                       which='major', 
                       length=8, 
                       color='black')
        
        ax.tick_params(axis='y', 
                       which='minor', 
                       length=4, 
                       color='black')
        
        ax.tick_params(axis='x', 
                       which='major', 
                       length=8, 
                       color='black')

        ax.tick_params(axis='x', 
                       which='minor', 
                       length=4, 
                       color='black')
            
    plt.savefig('./data/fig/Vis/profit_poten_'+ky+'.png',bbox_inches='tight')
    
    plt.close()